In [34]:
#!/usr/bin/env python
# coding: utf8
"""Example of training spaCy's named entity recognizer, starting off with an
existing model or a blank model.

For more details, see the documentation:
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities

Compatible with: spaCy v2.0.0+
Last tested with: v2.1.0
"""
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


# training data
#build a sentence extractor, iterate over them, pull out word/stop-start pairs, ask for a label, with option to manually
#adjust them for two word ones.
#grab Reddit comments: https://bigquery.cloud.google.com/dataset/fh-bigquery:reddit_comments?pli=1

TRAIN_DATA = [
    ("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
    ("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
    ("Boeing 737 MAX crashes a lot.", {"entities": [(0, 6, "ORG"), (7, 14, "PRODUCT")]}),
]


@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)

def main(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])


if __name__ == "__main__":
    main('en_core_web_md','data/nlp/custom_en')
    
    # Expected output:
    # Entities [('Shaka Khan', 'PERSON')]
    # Tokens [('Who', '', 2), ('is', '', 2), ('Shaka', 'PERSON', 3),
    # ('Khan', 'PERSON', 1), ('?', '', 2)]
    # Entities [('London', 'LOC'), ('Berlin', 'LOC')]
    # Tokens [('I', '', 2), ('like', '', 2), ('London', 'LOC', 3),
    # ('and', '', 2), ('Berlin', 'LOC', 3), ('.', '', 2)]

Loaded model 'en_core_web_md'
Losses {'ner': 8.374794286675751}
Losses {'ner': 6.722818702459335}
Losses {'ner': 8.302413940429688}
Losses {'ner': 6.911683481186628}
Losses {'ner': 6.6661085279192775}
Losses {'ner': 7.970680385828018}
Losses {'ner': 7.850625544786453}
Losses {'ner': 7.117354575544596}
Losses {'ner': 3.2527941912412643}
Losses {'ner': 8.190870718415681}
Losses {'ner': 5.393594918772578}
Losses {'ner': 6.603988409042358}
Losses {'ner': 2.58467850839952}
Losses {'ner': 3.221099464222789}
Losses {'ner': 3.683812853472773}
Losses {'ner': 2.248684511403553}
Losses {'ner': 3.975286907982081}
Losses {'ner': 5.3619325160980225}
Losses {'ner': 4.027989878813969}
Losses {'ner': 1.7940432704411933}
Losses {'ner': 1.1645340446266346}
Losses {'ner': 2.270657498855144}
Losses {'ner': 4.771896449383348}
Losses {'ner': 1.5128146522674797}
Losses {'ner': 1.0857081681051568}
Losses {'ner': 0.010558536101598293}
Losses {'ner': 0.4420203237564948}
Losses {'ner': 2.057387816719711}
Losses {